# The Viewer Component
The interactive viewer for IFC models has been extended from Thomas Paviots' excellent [JupyterRenderer](https://github.com/tpaviot/pythonocc-core/blob/master/src/Display/WebGl/jupyter_renderer.py) and offeres a number of functionalities:

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import ifcopenshell
import json
import collections
from ifcopenshell.util.selector import Selector
from utils.JupyterIFCRenderer import JupyterIFCRenderer


In [45]:
m = ifcopenshell.open("../FM.ifc")
system_dict_file = "../WestDeckBox.nwd_aggregation.json"
#m = ifcopenshell.open("data/231110AC-11-Smiley-West-04-07-2007.ifc")k

## Identify aggrgegation relationships

In [41]:

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        #print(type(v))
        if isinstance(v, list):
            for i, el in enumerate(v):
                new_key_2 = new_key + sep + str(i)
                if isinstance(el, collections.MutableMapping):
                    items.extend(flatten(el, new_key_2, sep=sep))
                else:
                    
                    items.append(el)
        elif isinstance(v, collections.MutableMapping):
            #print(v)
            items.extend(flatten(v, new_key, sep=sep))
        else:
            print(type(v))
            items.append(v)
    return (items)


def getSystems(system_dict_file):
    f = open(system_dict_file)
    system_dict = json.load(f)
    root = list(system_dict.keys())[0]
    out_dict = {}

    for sys in system_dict[root]:
        sys_name = list(sys.keys())[0]
        
        out_dict[sys_name] = flatten(sys)
        print(sys_name, len(out_dict[sys_name]))
        

getSystems(system_dict_file)    
    
    

West-DeckBox-Support.rvm 57062
West-DeckBox-Electrical.rvm 32334
West-DeckBox-Structure.rvm 19892
West-DeckBox-Instrumentation.rvm 37549
West-DeckBox-Pipe.rvm 33565
West-DeckBox-Mechanical.rvm 31107
West-DeckBox-Architecture.rvm 11181
West-DeckBox-Safety.rvm 2638
West-DeckBox-HVAC.rvm 7749


To invoke it we have to import the models residing in the local `utils` folder.

In [46]:
viewer = JupyterIFCRenderer(m, size=(400,300))
viewer

### colour by system

In [57]:
element_name = 'Rohrtypen:Kupfer - Hartgelötet:7718880'
selector = Selector()
elements = selector.parse(m, '.IfcProduct[Name *= "' + element_name + '"]')
print(elements)
colour = "#ff0000"

viewer.setColor(elements[0], colour)

[#217510=IfcPipeSegment('3ELBYKoMX93QrK7TjQe0eV',#42,'Rohrtypen:Kupfer - Hartgelötet:7718880',$,'Rohrtypen:Kupfer - Hartgelötet',#217494,#217507,'7718880',.NOTDEFINED.)]


Instances of building elements with represenations can be selected interactivly. Information such as the attributes `GUID`, `Name` etc. are displayed to the left of the 3D viewport.

In [48]:
viewer.setDefaultColors()

## Selecting instances 

The instance object currently selected in the 3D view can be assigned to a avariable using the `.getSelectedProduct()` method. 

Provding the plain varialbe name call sthe `Display()`-method of the instance and prints the line of the original ifc-Datei the  [SPFF-format](https://en.wikipedia.org/wiki/ISO_10303-21).


In [47]:
selection = viewer.getSelectedProduct()
selection

#217510=IfcPipeSegment('3ELBYKoMX93QrK7TjQe0eV',#42,'Rohrtypen:Kupfer - Hartgelötet:7718880',$,'Rohrtypen:Kupfer - Hartgelötet',#217494,#217507,'7718880',.NOTDEFINED.)

## Show / Hide Objects
Programatically or interactively selected projects can be hidden or shown in the viewer using `setVisible()`.


In [ ]:
viewer.setVisible(selection, False)
viewer

Opening elements are displayed by default. Let's hide them.

In [ ]:
for opening in m.by_type("IfcOpeningElement"):
    viewer.setVisible(opening, False)

## Coloring Objects
Objects can be colored either by hexadicimal numbers provided as strings as they are common in e.g. in HTML and CSS and can be retrieved from many ressources

In [55]:

viewer.setColorSelected("#ddffaa")
viewer

AttributeError: 'NoneType' object has no attribute 'material'

If you do not happen to have a color in mind  just call a color picker from the viewer:

In [53]:
picker = viewer.colorPicker()
picker

AttributeError: 'JupyterIFCRenderer' object has no attribute 'colorPicker'

In [ ]:
viewer.setColorSelected(picker.value)

### Default colors
a number of colors has been defined as a fallback palette and can be called using `viewer.setDefaultColors()`

In [ ]:
viewer.setDefaultColors()

In [ ]:
viewer